# Processing EK60 Data on AWS

- Creators: Rudy Klucik<sup>1</sup>, Veronica Martinez<sup>1</sup>, Charles Anderson<sup>1</sup>, and Carrie Wall<sup>1</sup>
- Affiliations: <sup>1</sup>Cooperative Institute for Research in Environmental Sciences ([CIRES](https://cires.colorado.edu/))
- History:
    - Version 1, 2024-09-27

## Overview

This tutorial demonstrates how to process Simrad EK60 (18-710 kHz, split beam) echosounder data in the cloud using [`echopype`](https://echopype.readthedocs.io/en/stable/), an open-source, python-based module for reading, processing, and visualizing water-column echosounder files developed by scientists at the University of Washington.

## Prerequisites
To successfully navigate and use this notebook, you should be familiar with:

- the basics of Python programming such as loading modules assigning variables, and list/array indexing
- plotting data using matplotlib

## Learning Outcomes
By working through this notebook, you will learn how to:

- access and read a raw EK60 file from an AWS S3 bucket
- find the calibrated volume backscattering strength (Sv)
- plot water column sonar data

## Time Estimate
- Estimated text reading time: 6 to 11 min
- Estimated code reading time: 2 to 4 min
- Estimated total reading time: 8 to 15 min

## Background

Water column sonar data, the acoustic back-scatter from the near-surface to the seafloor, are used to assess physical and biological characteristics of the ocean including the spatial distribution of plankton, fish, methane seeps, and underwater oil plumes. There are a variety of sonar systems including single and multibeam echosounders using single frequency, multiple frequencies, or wideband technology.

 These systems deliver valuable information for ecosystem-based fisheries management but they also produce large data volumes that are costly and complicated to maintain. In collaboration with NOAA Fisheries and CIRES, NCEI archives water column sonar data collected from NOAA, academic, and internationals fleets. Archived data are globally accessible through the archive's [data portal](https://www.ncei.noaa.gov/maps/water-column-sonar/) as well as on Amazon Web Services through the NOAA Open Data Dissemintation Program ([NODD](https://www.noaa.gov/information-technology/open-data-dissemination)).

 The native sonar data are large, complex, and recorded in instrument-specific binary file formats. Embedded in the native file's metadata and datagrams are a number of fields including sonar configuration information, navigation information, calibration coefficients and power data from single or multiple sonar frequencies. File names contain the start time for that file, and often include a preceding tag for that cruise or deployment. The timestamp in UTC follows the convention: ‘D’YYYYMMDD’-T’hhmmss. For example, “SaKe2015-D20150719-T193412”, indicates a file from a 2015 SaKe cruise and the start of the file is July 19, 2013 at 19:34:12 (UTC).

The largest volume of raw sonar data in the [NCEI archive](https://www.ncei.noaa.gov/products/water-column-sonar-data) were collected using Simrad EK60 (18-710 kHz, split beam) and EK80 (18-710 kHz, split beam and broadband) echosounders.

## Installing and importing libraries

We will first need to install a couple Python libraries for accessing and processing data. One library used for water column sonar data is `echopype`. The following code check if the libraries are already installed in your environment and install the missing libraries.

In [ ]:
# sys provides access to system level information (i.e., the executable for the Python installation)
import sys
# subprocess makes system level calls
import subprocess

# List of packages used in this notebook
PACKAGES = ["boto3", "botocore", "echopype"]

# Loop through each package to either import or install
for package in PACKAGES:
    try:
        # First, attempt to import the package
        __import__(package)
    except ImportError:
        # If package import is unsuccessful, install using pip
        # The command structure is <python executable> -m pip install <package>
        if package == 'echopype': package = 'echopype==0.7.1'
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])

Next import the libraries:

In [ ]:
import os
import boto3

import echopype.visualize as epviz
import matplotlib.pyplot as plt
import echopype as ep

#import warnings
#warnings.simplefilter("ignore", category=DeprecationWarning)

In [ ]:
print(f"echopype version: {ep.__version__}")

## Public Data

Publically available data can be accessed from an S3 bucket managed by NOAA's National Centers for Environmental Information. Over 250 TB of data are available for download.


More information on the NOAA Water-Column Sonar Data Archive can be found at the [registry of open data on AWS](https://registry.opendata.aws/ncei-wcsd-archive/)

Files can be easily explored using the [AWS bucket explorer.](https://noaa-wcsd-pds.s3.amazonaws.com/index.html#data/raw/Henry_B._Bigelow/HB0707/EK60/)

The format of the files we will process follow a URL format as follows:

> `https://<Bucket Name>.s3.amazonaws.com/data/raw/<Ship Name>/<Cruise Name>/<Sensor Name>/<File Name>`

For example:

> `https://noaa-wcsd-pds.s3.amazonaws.com/data/raw/Bell_M._Shimada/SH1507/EK60/SaKe2015-D20150719-T193412.raw`

And to learn more about [NCEI's Water Column Sonar Data Archive.](https://www.ncei.noaa.gov/products/water-column-sonar-data)

## Reading the Data

First, we will read data from an S3 bucket. The following code reads files passed to it, stores the data in an object, retrieves values from the data, and appends and inserts data from different sampled intervals.

In [ ]:
bucket_name = 'noaa-wcsd-pds'

ship_name = 'Bell_M._Shimada'
cruise_name = 'SH1507'
sensor_name = 'EK60'
file_names = [
  "SaKe2015-D20150719-T193412.raw",
  "SaKe2015-D20150719-T193443.raw",
  "SaKe2015-D20150719-T194042.raw",
  "SaKe2015-D20150719-T194642.raw",
  "SaKe2015-D20150719-T195242.raw",
  "SaKe2015-D20150719-T195842.raw"
]

The next step is to download the raw EK60 files from the S3 public dataset, 'noaa-wcsd-pds.' To do this we will use a library called "boto3." This is a library that gives you access to a full suite of AWS tools, we are just going to use it to download a file from an S3 bucket, and we will do so anonymously using the "UNSIGNED" configuration below.

In [ ]:
from botocore import UNSIGNED
from botocore.client import Config

s3 = boto3.client(service_name='s3', config=Config(signature_version=UNSIGNED))
for file_name in file_names:
  s3.download_file(
      Bucket=bucket_name,
      Key=f'data/raw/{ship_name}/{cruise_name}/{sensor_name}/{file_name}',
      Filename=os.path.basename(file_name),
  )
  print(os.path.basename(file_name))

Next verify that the files were downloaded locally by listing files in the current directory.

In [ ]:
os.listdir()

## Opening the files with echopype

[`Echopype`](https://echopype.readthedocs.io/en/stable/index.html) is a python package that enables processing of ocean sonar data. We will use this package to read and process the data files that were downloaded.

In [ ]:
echodata = []
for i in range(len(file_names)):
  print(f"Processing: {file_names[i]}")
  echodata.append(ep.open_raw(raw_file=file_names[i], sonar_model=sensor_name))

We can look at the structure of the data by referencing the first file as follows:

In [ ]:
echodata[0]

Echopype organizes the data similar to Xarray's Datasets. Subsets and DataArrays can be accessed via the dot operator, e.g. to access the echodata environment variables you can add ".environment" to the end of the variable.

You will see that an echodata environment variable is composed of:


*   Dimensions
*   Coordinates
*   Data Variables
*   Indexes
*   Attributes

In [ ]:
echodata[0].environment

Right now our echodata variable is just an array or individually opened echodata objects. Those objects can be combined to form a single echodata object as follows:

In [ ]:
# this code deletes any existing zarr files if you run this notebook more than once
!rm -rf /root/.echopype/temp_output/combined_echodata.zarr

In [ ]:
combined_echodata = ep.combine_echodata(echodata)

## Plot the Volume Backscattering Strength, Sv

Echopype has built in methods to easily plot data echodata. To learn more, read the documentation [here](https://echopype.readthedocs.io/en/stable/viz.html).

In [ ]:
epviz.create_echogram(combined_echodata, get_range=True, robust=True)

## Calibrate the Data

Volume backscattering strength (Sv) from raw data can be computed as follows:

In [ ]:
ds_Sv = ep.calibrate.compute_Sv(combined_echodata)

In [ ]:
ds_Sv

In [ ]:
ds_Sv.coords['channel']

Because the coordinates of an echopype Dataset are defined by "channel," we can swap the dimensions to organize them by "frequency" instead.

In [ ]:
ds_Sv_freq = ep.consolidate.swap_dims_channel_frequency(ds_Sv)

The updated "ds_Sv_freq" variable now has a coordinate axis defined by the ordinal "frequency_nominal" value versus the previous string-like "channel" as seen in the ds_Sv variable above. This allows data to be subsetted by "frequency" when plotting.

In [ ]:
ds_Sv_freq

In [ ]:
ds_Sv_freq.Sv.sel(frequency_nominal=18000).plot.pcolormesh(x='ping_time', cmap = 'jet', vmin=-80, vmax=-30, yincrease=False)

The data in the plot above shows a detected bottom at 19:19:35 that starts near 900 meters and tapers to the botom-right down to ~2,200 meters at 19:20:05. Also interesting to note is that beneath the seabed, two harmonics can also be observed which mirror the shape of the ocean floor.

If we would like to further subset the data, we can zoom into the area of interest, here that would entail selecting range_sample(s) between zero and ~2400 meters.

In [ ]:
ds_Sv_freq.Sv.sel(frequency_nominal=18000, range_sample=slice(0, 2400)).plot.pcolormesh(x='ping_time', cmap = 'jet', vmin=-80, vmax=-30, yincrease=False)

The same plot but selecting data from the 38 kHz frequency instead:

In [ ]:
ds_Sv_freq.Sv.sel(frequency_nominal=38000, range_sample=slice(0, 2400)).plot.pcolormesh(x='ping_time', cmap = 'jet', vmin=-80, vmax=-30, yincrease=False)

And the 70 kHz frequency:

In [ ]:
ds_Sv_freq.Sv.sel(frequency_nominal=70000, range_sample=slice(0, 2400)).plot.pcolormesh(x='ping_time', cmap = 'jet', vmin=-80, vmax=-30, yincrease=False)

The 120 kHz frequency:

In [ ]:
ds_Sv_freq.Sv.sel(frequency_nominal=120000, range_sample=slice(0, 2400)).plot.pcolormesh(x='ping_time', cmap = 'jet', vmin=-80, vmax=-30, yincrease=False)

And finally the 120 kHz frequency:

In [ ]:
ds_Sv_freq.Sv.sel(frequency_nominal=200000, range_sample=slice(0, 2400)).plot.pcolormesh(x='ping_time', cmap = 'jet', vmin=-80, vmax=-30, yincrease=False)

Note the change in characteristic of the data with the change in sonar frequency.

By accessing the data using the underlying Xarray Dataset methods, plots can be further configured and allow for diverse subsetting of the data.

# Data statement
All data used in this notebook are publicly available in the [NOAA NCEI Water Column Sonar Data Archive](https://www.ncei.noaa.gov/maps/water-column-sonar/index.html?surveys=SH1507&instruments=EK60)

- A copy can also be found in the archive's [NODD AWS bucket](https://noaa-wcsd-pds.s3.amazonaws.com/index.html#data/raw/Bell_M._Shimada/SH1507/EK60)

# Acknowledgments
Funding support was provided by the NOAA Center for Artificial Intelligence ([NCAI](https://www.noaa.gov/noaa-center-for-artificial-intelligence/)) and [NOAA Fisheries](https://www.fisheries.noaa.gov/)

# References


*   Lee, W., Mayorga, E., Setiawan, L., Majeed, I., Nguyen, K., & Staneva, V. (2021). Echopype: A Python library for interoperable and scalable processing of water column sonar data for biological information. arXiv preprint arXiv:2111.00187
* Lee, W., Nguyen, K., Setiawan, L., Mayorga, E., Reyes, B., Majeed, I., & Staneva, V. (2023). echopype (v0.8.1). Zenodo. https://doi.org/10.5281/zenodo.8312077
* National Centers for Environmental Information: Water Column Sonar Data Collection. National Centers for Environmental Information, NOAA. http://doi.org/10.7289/V5HT2M7C.



# Metadata
- Language / packages(s):
    - Language: Python
    - Packages: Boto, Echopype, Matplotlib

- Scientific domain:
    - Fisheries acoustics

- Application keywords:
    - Sonar processing

- Geophysical keywords
    - Fish
    - Zooplankton
    - Seafloor

# License
## Software and Content Description License
Software code created by U.S. Government employees is not subject to copyright in the United States (17 U.S.C. §105). The United States/Department of Commerce reserve all rights to seek and obtain copyright protection in countries other than the United States for Software authored in its entirety by the Department of Commerce. To this end, the Department of Commerce hereby grants to Recipient a royalty-free, nonexclusive license to use, copy, and create derivative works of the Software outside of the United States.




# Disclaimer
This Jupyter notebook is a scientific product and is not official communication of the National Oceanic and Atmospheric Administration, or the United States Department of Commerce. All NOAA Jupyter notebooks are provided on an 'as is' basis and the user assumes responsibility for its use. Any claims against the Department of Commerce or Department of Commerce bureaus stemming from the use of this Jupyter notebook will be governed by all applicable Federal law. Any reference to specific commercial products, processes, or services by service mark, trademark, manufacturer, or otherwise does not constitute or imply their endorsement, recommendation or favoring by the Department of Commerce. The Department of Commerce seal and logo, or the seal and logo of a DOC bureau, shall not be used in any manner to imply endorsement of any commercial product or activity by DOC or the United States Government.